**Note**: Click on "*Kernel*" > "*Restart Kernel and Run All*" in [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/) *after* finishing the exercises to ensure that your solution runs top to bottom *without* any errors. If you cannot run this file on your machine, you may want to open it [in the cloud <img height="12" style="display: inline-block" src="../static/link/to_mb.png">](https://mybinder.org/v2/gh/webartifex/intro-to-python/develop?urlpath=lab/tree/11_classes/01_exercises.ipynb).

# Chapter 11: Classes & Instances (Coding Exercises)

The exercises below assume that you have read the [first part <img height="12" style="display: inline-block" src="../static/link/to_nb.png">](https://nbviewer.jupyter.org/github/webartifex/intro-to-python/blob/develop/11_classes/00_content.ipynb) of Chapter 11.

The `...`'s in the code cells indicate where you need to fill in code snippets. The number of `...`'s within a code cell give you a rough idea of how many lines of code are needed to solve the task. You should not need to create any additional code cells for your final solution. However, you may want to use temporary code cells to try out some ideas.

## Berlin Tourist Guide: A Traveling Salesman Problem

This notebook is a hands-on and tutorial-like application to show how to load data from web services like [Google Maps](https://developers.google.com/maps) and use them to solve a logistics problem, namely a **[Traveling Salesman Problem <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Traveling_salesman_problem)**.

Imagine that a tourist lands at Berlin's [Tegel Airport <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Berlin_Tegel_Airport) in the morning and has his "connecting" flight from [Schönefeld Airport <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Berlin_Sch%C3%B6nefeld_Airport) in the evening. By the time, the flights were scheduled, the airline thought that there would be only one airport in Berlin.

Having never been in Berlin before, the tourist wants to come up with a plan of sights that he can visit with a rental car on his way from Tegel to Schönefeld.

With a bit of research, he creates a `list` of `sights` like below.

In [1]:
arrival = "Berlin Tegel Airport (TXL), Berlin"

sights = [
    "Alexanderplatz, Berlin",
    "Brandenburger Tor, Pariser Platz, Berlin",
    "Checkpoint Charlie, Friedrichstraße, Berlin",
    "Kottbusser Tor, Berlin",
    "Mauerpark, Berlin",
    "Siegessäule, Berlin",
    "Reichstag, Platz der Republik, Berlin",
    "Soho House Berlin, Torstraße, Berlin",
    "Tempelhofer Feld, Berlin",
]

departure = "Berlin Schönefeld Airport (SXF), Berlin"

With just the street addresses, however, he cannot calculate a route. He needs `latitude`-`longitude` coordinates instead. While he could just open a site like [Google Maps](https://www.google.com/maps) in a web browser, he wonders if he can download the data with a bit of Python code using a [web API <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Web_API) offered by [Google](https://www.google.com).

So, in this notebook, we solve the entire problem with code.

## Geocoding

In order to obtain coordinates for the given street addresses above, a process called **geocoding**, we use the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start).

**Q1**: Familiarize yourself with this [documentation](https://developers.google.com/maps/documentation/geocoding/start), register a developer account, create a project, and [create an API key](https://console.cloud.google.com/apis/credentials) that is necessary for everything to work! Then, [enable the Geocoding API](https://console.developers.google.com/apis/library/geocoding-backend.googleapis.com) and link a [billing account](https://console.developers.google.com/billing)!

Info: The first 200 Dollars per month are not charged (cf., [pricing page](https://cloud.google.com/maps-platform/pricing/)), so no costs will incur for this tutorial. You must sign up because Google simply wants to know the people using its services.

**Q2**: Assign the API key as a `str` object to the `key` variable!

In [2]:
key = " < your API key goes here > "

To use external web services, our application needs to make HTTP requests just like web browsers do when surfing the web.

We do not have to implement this on our own. Instead, we use the official Python Client for the Google Maps Services provided by Google in one of its corporate [GitHub repositories <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/googlemaps).

**Q3**: Familiarize yourself with the [googlemaps <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/googlemaps/google-maps-services-python) package! Then, install it with the `pip` command line tool!

In [3]:
!pip install googlemaps

**Q4**: Finish the following code cells and instantiate a `Client` object named `api`! Use the `key` from above. `api` provides us with a lot of methods to talk to the API.

In [4]:
import googlemaps

In [5]:
api = googlemaps.Client(key=key)

In [6]:
api

In [7]:
type(api)

googlemaps.client.Client

**Q5**: Execute the next code cell to list the methods and attributes on the `api` object!

In [8]:
[x for x in dir(api) if not x.startswith("_")]

['base_url',
 'channel',
 'clear_experience_id',
 'client_id',
 'client_secret',
 'directions',
 'distance_matrix',
 'elevation',
 'elevation_along_path',
 'find_place',
 'geocode',
 'geolocate',
 'get_experience_id',
 'key',
 'nearest_roads',
 'place',
 'places',
 'places_autocomplete',
 'places_autocomplete_query',
 'places_nearby',
 'places_photo',
 'queries_per_second',
 'requests_kwargs',
 'retry_over_query_limit',
 'retry_timeout',
 'reverse_geocode',
 'sent_times',
 'session',
 'set_experience_id',
 'snap_to_roads',
 'snapped_speed_limits',
 'speed_limits',
 'static_map',
 'timeout',
 'timezone']

To obtain all kinds of information associated with a street address, we call the `geocode()` method with the address as the sole argument.

For example, let's search for Brandenburg Gate. Its street address is `"Brandenburger Tor, Pariser Platz, Berlin"`.

**Q6**: Execute the next code cell!

Hint: If you get an error message, follow the instructions in it to debug it.

If everything works, we receive a `list` with a single `dict` in it. That means the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start) only knows about one place at the address. Unfortunately, the `dict` is pretty dense and hard to read.

In [9]:
api.geocode("Brandenburger Tor, Pariser Platz, Berlin")

[{'address_components': [{'long_name': 'Pariser Platz',
    'short_name': 'Pariser Platz',
    'types': ['route']},
   {'long_name': 'Mitte',
    'short_name': 'Mitte',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Berlin',
    'short_name': 'Berlin',
    'types': ['locality', 'political']},
   {'long_name': 'Berlin',
    'short_name': 'Berlin',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Germany',
    'short_name': 'DE',
    'types': ['country', 'political']},
   {'long_name': '10117', 'short_name': '10117', 'types': ['postal_code']}],
  'formatted_address': 'Pariser Platz, 10117 Berlin, Germany',
  'geometry': {'location': {'lat': 52.5162746, 'lng': 13.3777041},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 52.51762358029149,
     'lng': 13.3790530802915},
    'southwest': {'lat': 52.5149256197085, 'lng': 13.3763551197085}}},
  'place_id': 'ChIJiQnyVcZRqEcRY0xnhE77uyY',
  'plus_cod

**Q7**: Capture the first and only search result in the `brandenburg_gate` variable and "pretty print" it with the help of the [pprint() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/pprint.html#pprint.pprint) function in the [pprint <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/pprint.html) module in the [standard library <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/index.html)!

In [10]:
response = api.geocode("Brandenburger Tor, Pariser Platz, Berlin")

In [11]:
brandenburg_gate = response[0]

The `dict` has several keys that are of use for us: `"formatted_address"` is a cleanly formatted version of the address. `"geometry"` is a nested `dict` with several `lat`-`lng` coordinates representing the place where `"location"` is the one we need for our calculations. Lastly, `"place_id"` is a unique identifier that allows us to obtain further information about the address from other Google APIs.

In [12]:
from pprint import pprint

In [13]:
pprint(brandenburg_gate)

{'address_components': [{'long_name': 'Pariser Platz',
                         'short_name': 'Pariser Platz',
                         'types': ['route']},
                        {'long_name': 'Mitte',
                         'short_name': 'Mitte',
                         'types': ['political',
                                   'sublocality',
                                   'sublocality_level_1']},
                        {'long_name': 'Berlin',
                         'short_name': 'Berlin',
                         'types': ['locality', 'political']},
                        {'long_name': 'Berlin',
                         'short_name': 'Berlin',
                         'types': ['administrative_area_level_1', 'political']},
                        {'long_name': 'Germany',
                         'short_name': 'DE',
                         'types': ['country', 'political']},
                        {'long_name': '10117',
                         'short_name': '10117',
   

### The `Place` Class

To keep our code readable and maintainable, we create a `Place` class to manage the API results in a clean way.

The `.__init__()` method takes a `street_address` (e.g., an element of `sights`) and a `client` argument (e.g., an object like `api`) and stores them on `self`. The place's `.name` is parsed out of the `street_address` as well: It is the part before the first comma. Also, the instance attributes `.latitude`, `.longitude`, and `.place_id` are initialized to `None`.

**Q8**: Finish the `.__init__()` method according to the description!

The `.sync_from_google()` method uses the internally kept `client` and synchronizes the place's state with the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start). In particular, it updates the `.address` with the `formatted_address` and stores the values for `.latitude`, `.longitude`, and `.place_id`. It enables method chaining.

**Q9**: Implement the `.sync_from_google()` method according to the description!

**Q10**: Add a read-only `.location` property on the `Place` class that returns the `.latitude` and `.longitude` as a `tuple`!

In [14]:
class Place:
    """A place connected to the Google Maps Geocoding API."""

    # answer to Q8
    def __init__(self, street_address, *, client):
        """Create a new place.

        Args:
            street_address (str): street address of the place
            client (googlemaps.Client): access to the Google Maps Geocoding API
        """
        self.name = street_address.split(",")[0]
        self.address = street_address
        self.client = client
        self.latitude = None
        self.longitude = None
        self.place_id = None

    def __repr__(self):
        cls, name = self.__class__.__name__, self.name
        synced = " [SYNCED]" if self.place_id else ""
        return f"<{cls}: {name}{synced}>"

    # answer to Q9
    def sync_from_google(self):
        """Download the place's coordinates and other info."""
        response = self.client.geocode(self.address)
        first_hit = response[0]
        self.address = first_hit["formatted_address"]
        self.latitude = first_hit["geometry"]["location"]["lat"]
        self.longitude = first_hit["geometry"]["location"]["lng"]
        self.place_id = first_hit["place_id"]
        return self

    # answer to Q10
    @property
    def location(self):
        return self.latitude, self.longitude

**Q11**: Verify that the instantiating a `Place` object works!

In [15]:
brandenburg_gate = Place("Brandenburger Tor, Pariser Platz, Berlin", client=api)

In [16]:
brandenburg_gate

<Place: Brandenburger Tor>

**Q12**: What do the angle brackets `<` and `>` mean in the text representation?

 < your answer >

Now, we can obtain the geo-data from the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start) in a clean way. As we enabled method chaining for `.sync_from_google()`, we get back the instance after calling the method.

**Q13**: Verify that the `.sync_from_google()` method works!

In [17]:
brandenburg_gate.sync_from_google()

<Place: Brandenburger Tor [SYNCED]>

In [18]:
brandenburg_gate.address

'Pariser Platz, 10117 Berlin, Germany'

In [19]:
brandenburg_gate.place_id

'ChIJiQnyVcZRqEcRY0xnhE77uyY'

In [20]:
brandenburg_gate.location

(52.5162746, 13.3777041)

### The `Place` Class (continued): Batch  Synchronization

**Q14**: Add an alternative constructor method named `.from_addresses()` that takes an `addresses`, a `client`, and a `sync` argument! `addresses` is a finite iterable of `str` objects (e.g., like `sights`). The method returns a `list` of `Place`s, one for each `str` in `addresses`. All `Place`s are initialized with the same `client`. `sync` is a flag and defaults to `False`. If it is set to `True`, the alternative constructor invokes the `.sync_from_google()` method on the `Place`s before returning them.

In [21]:
class Place:
    """A place connected to the Google Maps Geocoding API."""

    # answers from above
    def __init__(self, street_address, *, client):
        """Create a new place.

        Args:
            street_address (str): street address of the place
            client (googlemaps.Client): access to the Google Maps Geocoding API
        """
        self.name = street_address.split(",")[0]
        self.address = street_address
        self.client = client
        self.latitude = None
        self.longitude = None
        self.place_id = None

    def __repr__(self):
        cls, name = self.__class__.__name__, self.name
        synced = " [SYNCED]" if self.place_id else ""
        return f"<{cls}: {name}{synced}>"

    def sync_from_google(self):
        """Download the place's coordinates and other info."""
        response = self.client.geocode(self.address)
        first_hit = response[0]
        self.address = first_hit["formatted_address"]
        self.latitude = first_hit["geometry"]["location"]["lat"]
        self.longitude = first_hit["geometry"]["location"]["lng"]
        self.place_id = first_hit["place_id"]
        return self

    @property
    def location(self):
        return self.latitude, self.longitude

    # answer to Q14
    @classmethod
    def from_addresses(cls, addresses, *, client, sync=False):
        """Create new places in a batch.

        Args:
            addresses (iterable of str's): the street addresses of the places
            client (googlemaps.Client): access to the Google Maps Geocoding API
        Returns:
            list of Places
        """
        places = []
        for address in addresses:
            place = cls(address, client=client)
            if sync:
                place.sync_from_google()
            places.append(place)
        return places

**Q15**: Verify that the alternative constructor works with and without the `sync` flag!

In [22]:
Place.from_addresses(sights, client=api)

[<Place: Alexanderplatz>,
 <Place: Brandenburger Tor>,
 <Place: Checkpoint Charlie>,
 <Place: Kottbusser Tor>,
 <Place: Mauerpark>,
 <Place: Siegessäule>,
 <Place: Reichstag>,
 <Place: Soho House Berlin>,
 <Place: Tempelhofer Feld>]

In [23]:
Place.from_addresses(sights, client=api, sync=True)

[<Place: Alexanderplatz [SYNCED]>,
 <Place: Brandenburger Tor [SYNCED]>,
 <Place: Checkpoint Charlie [SYNCED]>,
 <Place: Kottbusser Tor [SYNCED]>,
 <Place: Mauerpark [SYNCED]>,
 <Place: Siegessäule [SYNCED]>,
 <Place: Reichstag [SYNCED]>,
 <Place: Soho House Berlin [SYNCED]>,
 <Place: Tempelhofer Feld [SYNCED]>]

## Visualization

For geo-data it always makes sense to plot them on a map. We use the third-party library [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium) to achieve that.

**Q16**: Familiarize yourself with [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium) and install it with the `pip` command line tool!

In [24]:
!pip install folium

**Q17**: Execute the code cells below to create an empty map of Berlin!

In [25]:
import folium

In [26]:
berlin = folium.Map(location=(52.513186, 13.3944349), zoom_start=14)

In [27]:
type(berlin)

folium.folium.Map

`folium.Map` instances are shown as interactive maps in Jupyter notebooks whenever they are the last expression in a code cell.

In [28]:
berlin

In order to put something on the map, [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium) works with so-called `Marker` objects.

**Q18**: Review its docstring and then create a marker `m` with the location data of Brandenburg Gate! Use the `brandenburg_gate` object from above!

Hint: You may want to use HTML tags for the `popup` argument to format the text output on the map in a nicer way. So, instead of just passing `"Brandenburger Tor"` as the `popup` argument, you could use, for example, `"<b>Brandenburger Tor</b><br/>(Pariser Platz, 10117 Berlin, Germany)"`. Then, the name appears in bold and the street address is put on the next line. You could use an f-string to parametrize the argument.

In [29]:
folium.Marker?

Init signature:
folium.Marker(
    location,
    popup=None,
    tooltip=None,
    icon=None,
    draggable=False,
    **kwargs,
)
Docstring:     
Create a simple stock Leaflet marker on the map, with optional
popup text or Vincent visualization.

Parameters
----------
location: tuple or list
    Latitude and Longitude of Marker (Northing, Easting)
popup: string or folium.Popup, default None
    Label for the Marker; either an escaped HTML string to initialize
    folium.Popup or a folium.Popup instance.
tooltip: str or folium.Tooltip, default None
    Display a text when hovering over the object.
icon: Icon plugin
    the Icon plugin to use to render the marker.
draggable: bool, default False
    Set to True to be able to drag the marker around the map.

Returns
-------
Marker names and HTML in obj.template_vars

Examples
--------
>>> Marker(location=[45.5, -122.3], popup='Portland, OR')
>>> Marker(location=[45.5, -122.3], popup=Popup('Portland, OR'))
# If the popup label has characte

In [30]:
m = folium.Marker(
    location=brandenburg_gate.location,
    popup=f"<b>{brandenburg_gate.name}</b><br/>({brandenburg_gate.address})",
    tooltip=brandenburg_gate.name,
)

In [31]:
type(m)

folium.map.Marker

**Q19**: Execute the next code cells that add `m` to the `berlin` map!

In [32]:
m.add_to(berlin)

In [33]:
berlin

### The `Place` Class (continued): Marker Representation

**Q20**: Finish the `.as_marker()` method that returns a `Marker` instance when invoked on a `Place` instance! The method takes an optional `color` argument that uses [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium)'s `Icon` type to control the color of the marker.

In [34]:
class Place:
    """A place connected to the Google Maps Geocoding API."""

    # answers from above
    def __init__(self, street_address, *, client):
        """Create a new place.

        Args:
            street_address (str): street address of the place
            client (googlemaps.Client): access to the Google Maps Geocoding API
        """
        self.name = street_address.split(",")[0]
        self.address = street_address
        self.client = client
        self.latitude = None
        self.longitude = None
        self.place_id = None

    def __repr__(self):
        cls, name = self.__class__.__name__, self.name
        synced = " [SYNCED]" if self.place_id else ""
        return f"<{cls}: {name}{synced}>"

    def sync_from_google(self):
        """Download the place's coordinates and other info."""
        response = self.client.geocode(self.address)
        first_hit = response[0]
        self.address = first_hit["formatted_address"]
        self.latitude = first_hit["geometry"]["location"]["lat"]
        self.longitude = first_hit["geometry"]["location"]["lng"]
        self.place_id = first_hit["place_id"]
        return self

    @property
    def location(self):
        return self.latitude, self.longitude

    @classmethod
    def from_addresses(cls, addresses, *, client, sync=False):
        """Create new places in a batch.

        Args:
            addresses (iterable of str's): the street addresses of the places
            client (googlemaps.Client): access to the Google Maps Geocoding API
        Returns:
            list of Places
        """
        places = []
        for address in addresses:
            place = cls(address, client=client)
            if sync:
                place.sync_from_google()
            places.append(place)
        return places

    # answer to Q20
    def as_marker(self, *, color="blue"):
        """Create a Marker representation of the place.

        Args:
            color (str): color of the marker, defaults to "blue"

        Returns:
            marker (folium.Marker)

        Raises:
            RuntimeError: if the place is not synchronized with
                the Google Maps Geocoding API
        """
        if not self.place_id:
            raise RuntimeError("must synchronize with Google first")
        return folium.Marker(
            location=self.location,
            popup=f"<b>{self.name}</b><br/>({self.address})",
            tooltip=self.name,
            icon=folium.Icon(color=color),
        )

**Q21**: Execute the next code cells that create a new `Place` and obtain a `Marker` for it!

Notes: Without synchronization, we get a `RuntimeError`. `.as_marker()` can be chained right after `.sync_from_google()`

In [35]:
brandenburg_gate = Place("Brandenburger Tor, Pariser Platz, Berlin", client=api)

In [36]:
brandenburg_gate.as_marker()

RuntimeError: must synchronize with Google first

In [37]:
brandenburg_gate.sync_from_google().as_marker()

**Q22**: Use the alternative `.from_addresses()` constructor to create a `list` named `places` with already synced `Place`s!

In [38]:
places = Place.from_addresses(sights, client=api, sync=True)

In [39]:
places

[<Place: Alexanderplatz [SYNCED]>,
 <Place: Brandenburger Tor [SYNCED]>,
 <Place: Checkpoint Charlie [SYNCED]>,
 <Place: Kottbusser Tor [SYNCED]>,
 <Place: Mauerpark [SYNCED]>,
 <Place: Siegessäule [SYNCED]>,
 <Place: Reichstag [SYNCED]>,
 <Place: Soho House Berlin [SYNCED]>,
 <Place: Tempelhofer Feld [SYNCED]>]

### The `Map` Class

To make [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium)'s `Map` class work even better with our `Place` instances, we write our own `Map` class wrapping [folium <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/python-visualization/folium)'s. Then, we add further functionality to the class throughout this tutorial.

The `.__init__()` method takes mandatory `name`, `center`, `start`, `end`, and `places` arguments. `name` is there for convenience, `center` is the map's initial center, `start` and `end` are `Place` instances, and `places` is a finite iterable of `Place` instances. Also, `.__init__()` accepts an optional `initial_zoom` argument defaulting to `12`.

Upon initialization, a `folium.Map` instance is created and stored as an implementation detail `_map`. Also, `.__init__()` puts markers for each place on the `_map` object: `"green"` and `"red"` markers for the `start` and `end` locations and `"blue"` ones for the `places` to be visited. To do that, `.__init__()` invokes another `.add_marker()` method on the `Map` class, once for every `Place` object. `.add_marker()` itself invokes the `.add_to()` method on the `folium.Marker` representation of a `Place` instance and enables method chaining.

To keep the state in a `Map` instance consistent, all passed in arguments except `name` are treated as implementation details. Otherwise, a user of the `Map` class could, for example, change the `start` attribute, which would not be reflected in the internally kept `folium.Map` object.

**Q23**: Implement the `.__init__()` and `.add_marker()` methods on the `Map` class as described!

**Q24**: Add a `.show()` method on the `Map` class that simply returns the internal `folium.Map` object!

In [40]:
class Map:
    """A map with plotting and routing capabilities."""

    # answer to Q23
    def __init__(self, name, center, start, end, places, initial_zoom=12):
        """Create a new map.

        Args:
            name (str): name of the map
            center (float, float): coordinates of the map's center
            start (Place): start of the tour
            end (Place): end of the tour
            places (iterable of Places): the places to be visitied
            initial_zoom (integer): zoom level according to folium's
                specifications; defaults to 12
        """
        self.name = name
        self._start = start
        self._end = end
        self._places = places
        self._map = folium.Map(location=center, zoom_start=initial_zoom)

        # Add markers to the map.
        self.add_marker(start.as_marker(color="green"))
        self.add_marker(end.as_marker(color="red"))
        for place in places:
            self.add_marker(place.as_marker())

    def __repr__(self):
        return f"<Map: {self.name}>"

    # answer to Q24
    def show(self):
        """Return a folium.Map representation of the map."""
        return self._map

    # answer to Q23
    def add_marker(self, marker):
        """Add a marker to the map.

        Args:
            marker (folium.Marker): marker to be put on the map
        """
        marker.add_to(self._map)
        return self

Let's put all the sights, the two airports, and three more places, the [Bundeskanzleramt <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/German_Chancellery), the [Olympic Stadium <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Olympiastadion_%28Berlin%29), and the [East Side Gallery <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/East_Side_Gallery), on the map.

**Q25**: Execute the next code cells to create a map of Berlin with all the places on it!

Note: Because we implemented method chaining everywhere, the code below is only *one* expression written over several lines. It almost looks like a self-explanatory and compact "language" on its own.

In [41]:
berlin = (
    Map(
        "Sights in Berlin",
        center=(52.5015154, 13.4066838),
        start=Place(arrival, client=api).sync_from_google(),
        end=Place(departure, client=api).sync_from_google(),
        places=places,
        initial_zoom=10,
    )
    .add_marker(
        Place("Bundeskanzleramt, Willy-Brandt-Straße, Berlin", client=api)
        .sync_from_google()
        .as_marker(color="orange")
    )
    .add_marker(
        Place("Olympiastadion Berlin", client=api)
        .sync_from_google()
        .as_marker(color="orange")
    )
    .add_marker(
        Place("East Side Gallery, Berlin", client=api)
        .sync_from_google()
        .as_marker(color="orange")
    )
)

In [42]:
berlin

<Map: Sights in Berlin>

In [43]:
berlin.show()

## Distance Matrix

Before we can find out the best order in which to visit all the sights, we must calculate the pairwise distances between all points. While Google also offers a [Directions API](https://developers.google.com/maps/documentation/directions/start) and a [Distance Matrix API](https://developers.google.com/maps/documentation/distance-matrix/start), we choose to calculate the air distances using the third-party library [geopy <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/geopy/geopy).

**Q26**: Familiarize yourself with the [documentation](https://geopy.readthedocs.io/en/stable/) and install [geopy <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/geopy/geopy) with the `pip` command line tool!

In [44]:
!pip install geopy

We use [geopy <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/geopy/geopy) primarily for converting the `latitude`-`longitude` coordinates into a [distance matrix <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Distance_matrix).

Because the [earth is not flat <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Flat_Earth), [geopy <img height="12" style="display: inline-block" src="../static/link/to_gh.png">](https://github.com/geopy/geopy) provides a `great_circle()` function that calculates the so-called [orthodromic distance <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Great-circle_distance) between two places on a sphere.

In [45]:
from geopy.distance import great_circle

**Q27**: For quick reference, read the docstring of `great_circle()` and execute the code cells below to calculate the distance between the `arrival` and the `departure`!

In [46]:
great_circle?

Init signature: great_circle(*args, **kwargs)
Docstring:     
Use spherical geometry to calculate the surface distance between two
points.

Set which radius of the earth to use by specifying a ``radius`` keyword
argument. It must be in kilometers. The default is to use the module
constant `EARTH_RADIUS`, which uses the average great-circle radius.

Example::

    >>> from geopy.distance import great_circle
    >>> newport_ri = (41.49008, -71.312796)
    >>> cleveland_oh = (41.499498, -81.695391)
    >>> print(great_circle(newport_ri, cleveland_oh).miles)
    536.997990696
File:           ~/repos/intro-to-python/.venv/lib/python3.8/site-packages/geopy/distance.py
Type:           ABCMeta
Subclasses:     


In [47]:
tegel = Place(arrival, client=api).sync_from_google()
schoenefeld = Place(departure, client=api).sync_from_google()

In [48]:
great_circle(tegel.location, schoenefeld.location)

Distance(25.370448366418135)

In [49]:
great_circle(tegel.location, schoenefeld.location).km

25.370448366418135

In [50]:
great_circle(tegel.location, schoenefeld.location).meters

25370.448366418135

### The `Place` Class (continued): Distance to another `Place`

**Q28**: Finish the `distance_to()` method in the `Place` class that takes a `other` argument and returns the distance in meters! Adhere to the given docstring!

In [51]:
class Place:
    """A place connected to the Google Maps Geocoding API."""

    # answers from above
    def __init__(self, street_address, *, client):
        """Create a new place.

        Args:
            street_address (str): street address of the place
            client (googlemaps.Client): access to the Google Maps Geocoding API
        """
        self.name = street_address.split(",")[0]
        self.address = street_address
        self.client = client
        self.latitude = None
        self.longitude = None
        self.place_id = None

    def __repr__(self):
        cls, name = self.__class__.__name__, self.name
        synced = " [SYNCED]" if self.place_id else ""
        return f"<{cls}: {name}{synced}>"

    def sync_from_google(self):
        """Download the place's coordinates and other info."""
        response = self.client.geocode(self.address)
        first_hit = response[0]
        self.address = first_hit["formatted_address"]
        self.latitude = first_hit["geometry"]["location"]["lat"]
        self.longitude = first_hit["geometry"]["location"]["lng"]
        self.place_id = first_hit["place_id"]
        return self

    @property
    def location(self):
        return self.latitude, self.longitude

    @classmethod
    def from_addresses(cls, addresses, *, client, sync=False):
        """Create new places in a batch.

        Args:
            addresses (iterable of str's): the street addresses of the places
            client (googlemaps.Client): access to the Google Maps Geocoding API
        Returns:
            list of Places
        """
        places = []
        for address in addresses:
            place = cls(address, client=client)
            if sync:
                place.sync_from_google()
            places.append(place)
        return places

    def as_marker(self, color="blue"):
        """Create a Marker representation of the place.

        Args:
            color (str): color of the marker, defaults to "blue"

        Returns:
            marker (folium.Marker)

        Raises:
            RuntimeError: if the place is not synchronized with
                the Google Maps Geocoding API
        """
        if not self.place_id:
            raise RuntimeError("must synchronize with Google first")
        return folium.Marker(
            location=self.location,
            popup=f"<b>{self.name}</b><br/>({self.address})",
            tooltip=self.name,
            icon=folium.Icon(color=color),
        )

    # answer to Q28
    def distance_to(self, other):
        """Calculate the distance to another place in meters.

        Args:
            other (Place): the other place to calculate the distance to

        Returns:
            distance (int)

        Raises:
            RuntimeError: if one of the places is not synchronized with
                the Google Maps Geocoding API
        """
        if not self.place_id or not other.place_id:
            raise RuntimeError("must synchronize places with Google first")
        return int(great_circle(self.location, other.location).meters)

**Q29**: Execute the code cells below to test the new feature!

Note: If done right, object-oriented code reads almost like plain English.

In [52]:
tegel = Place(arrival, client=api).sync_from_google()
schoenefeld = Place(departure, client=api).sync_from_google()

In [53]:
tegel.distance_to(schoenefeld)

25370

**Q30**: Execute the next code cell to instantiate the `Place`s in `sights` again!

In [54]:
places = Place.from_addresses(sights, client=api, sync=True)

In [55]:
places

[<Place: Alexanderplatz [SYNCED]>,
 <Place: Brandenburger Tor [SYNCED]>,
 <Place: Checkpoint Charlie [SYNCED]>,
 <Place: Kottbusser Tor [SYNCED]>,
 <Place: Mauerpark [SYNCED]>,
 <Place: Siegessäule [SYNCED]>,
 <Place: Reichstag [SYNCED]>,
 <Place: Soho House Berlin [SYNCED]>,
 <Place: Tempelhofer Feld [SYNCED]>]

### The `Map` Class (continued): Pairwise Distances

Now, we add a read-only `distances` property on our `Map` class. As we are working with air distances, these are *symmetric* which reduces the number of distances we must calculate.

To do so, we use the [combinations() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html#itertools.combinations) generator function in the [itertools <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html) module in the [standard library <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/index.html). That produces all possible `r`-`tuple`s from an `iterable` argument. `r` is `2` in our case as we are looking at `origin`-`destination` pairs.

Let's first look at an easy example of [combinations() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html#itertools.combinations) to understand how it works: It gives us all the `2`-`tuple`s from a `list` of five `numbers` disregarding the order of the `tuple`s' elements.

In [56]:
import itertools

In [57]:
numbers = [1, 2, 3, 4, 5]

for x, y in itertools.combinations(numbers, 2):
    print(x, y)

1 2
1 3
1 4
1 5
2 3
2 4
2 5
3 4
3 5
4 5


`distances` uses the internal `._start`, `._end`, and `._places` attributes and creates a `dict` with the keys consisting of all pairs of `Place`s and the values being their distances in meters. As this operation is rather costly, we cache the distances the first time we calculate them into a hidden instance attribute `._distances`, which must be initialized with `None` in the `.__init__()` method.

**Q31**: Finish the `.distances` property as described!

In [58]:
class Map:
    """A map with plotting and routing capabilities."""

    # answers from above with a tiny adjustment
    def __init__(self, name, center, start, end, places, initial_zoom=12):
        """Create a new map.

        Args:
            name (str): name of the map
            center (float, float): coordinates of the map's center
            start (Place): start of the tour
            end (Place): end of the tour
            places (iterable of Places): the places to be visitied
            initial_zoom (integer): zoom level according to folium's
                specifications; defaults to 12
        """
        self.name = name
        self._start = start
        self._end = end
        self._places = places
        self._map = folium.Map(location=center, zoom_start=initial_zoom)
        self._distances = None

        self.add_marker(start.as_marker(color="green"))
        self.add_marker(end.as_marker(color="red"))
        for place in places:
            self.add_marker(place.as_marker())

    def __repr__(self):
        return f"<Map: {self.name}>"

    def show(self):
        """Return a folium.Map representation of the map."""
        return self._map

    def add_marker(self, marker):
        """Add a marker to the map.

        Args:
            marker (folium.Marker): marker to be put on the map
        """
        marker.add_to(self._map)
        return self

    # answer to Q31
    @property
    def distances(self):
        """Return a dict with the pairwise distances of all places.

        Implementation note: The results of the calculations are cached.
        """
        if not self._distances:
            distances = {}
            all_pairs = itertools.combinations(
                [self._start, self._end] + self._places,
                r=2,
            )
            for origin, destination in all_pairs:
                distance = origin.distance_to(destination)
                distances[origin, destination] = distance
                distances[destination, origin] = distance
            self._distances = distances
        return self._distances

We pretty print the total distance matrix.

In [59]:
berlin = Map(
    "Berlin",
    center=(52.5015154, 13.4066838),
    start=Place(arrival, client=api).sync_from_google(),
    end=Place(departure, client=api).sync_from_google(),
    places=places,
    initial_zoom=10,
)

In [60]:
pprint(berlin.distances)

{(<Place: Berlin Tegel Airport (TXL) [SYNCED]>, <Place: Brandenburger Tor [SYNCED]>): 7671,
 (<Place: Siegessäule [SYNCED]>, <Place: Soho House Berlin [SYNCED]>): 4654,
 (<Place: Tempelhofer Feld [SYNCED]>, <Place: Siegessäule [SYNCED]>): 5589,
 (<Place: Reichstag [SYNCED]>, <Place: Siegessäule [SYNCED]>): 1821,
 (<Place: Alexanderplatz [SYNCED]>, <Place: Berlin Tegel Airport (TXL) [SYNCED]>): 9386,
 (<Place: Berlin Tegel Airport (TXL) [SYNCED]>, <Place: Berlin Schönefeld Airport (SXF) [SYNCED]>): 25370,
 (<Place: Berlin Tegel Airport (TXL) [SYNCED]>, <Place: Alexanderplatz [SYNCED]>): 9386,
 (<Place: Berlin Schönefeld Airport (SXF) [SYNCED]>, <Place: Berlin Tegel Airport (TXL) [SYNCED]>): 25370,
 (<Place: Soho House Berlin [SYNCED]>, <Place: Siegessäule [SYNCED]>): 4654,
 (<Place: Soho House Berlin [SYNCED]>, <Place: Tempelhofer Feld [SYNCED]>): 5878,
 (<Place: Reichstag [SYNCED]>, <Place: Soho House Berlin [SYNCED]>): 2839,
 (<Place: Reichstag [SYNCED]>, <Place: Tempelhofer Feld [SYN

How can we be sure the matrix contains all possible pairs? As we have 9 `sights` plus the `start` and the `end` of the tour, we conclude that there must be `11 * 10 = 110` distances excluding the `0` distances of a `Place` to itself that are not in the distance matrix.

In [61]:
n_places = len(places) + 2

n_places * (n_places - 1)

110

In [62]:
len(berlin.distances)

110

## Route Optimization

Let us find the cost minimal order of traveling from the `arrival` airport to the `departure` airport while visiting all the `sights`.

This problem can be expressed as finding the shortest so-called [Hamiltonian path <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Hamiltonian_path) from the `start` to `end` on the `Map` (i.e., a path that visits each intermediate node exactly once). With the "hack" of assuming the distance of traveling from the `end` to the `start` to be `0` and thereby effectively merging the two airports into a single node, the problem can be viewed as a so-called [traveling salesman problem <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Traveling_salesman_problem) (TSP).

The TSP is a hard problem to solve but also well studied in the literature. Assuming symmetric distances, a TSP with $n$ nodes has $\frac{(n-1)!}{2}$ possible routes. $(n-1)$ because any node can be the `start` / `end` and divided by $2$ as the problem is symmetric.

Starting with about $n = 20$, the TSP is almost impossible to solve exactly in a reasonable amount of time. Luckily, we do not have that many `sights` to visit, and so we use a [brute force <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Brute-force_search) approach and simply loop over all possible routes to find the shortest.

In the case of our tourist, we "only" need to try out `181_440` possible routes because the two airports are effectively one node and $n$ becomes $10$.

In [63]:
import math

In [64]:
math.factorial(len(places) + 1 - 1) // 2

181440

Analyzing the problem a bit further, all we need is a list of [permutations <img height="12" style="display: inline-block" src="../static/link/to_wiki.png">](https://en.wikipedia.org/wiki/Permutation) of the sights as the two airports are always the first and last location.

The [permutations() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html#itertools.permutations) generator function in the [itertools <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html) module in the [standard library <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/index.html) helps us with the task. Let's see an example to understand how it works.

In [65]:
numbers = [1, 2, 3]

for permutation in itertools.permutations(numbers):
    print(permutation)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


However, if we use [permutations() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html#itertools.permutations) as is, we try out *redundant* routes. For example, transferred to our case, the tuples `(1, 2, 3)` and `(3, 2, 1)` represent the *same* route as the distances are symmetric and the tourist could be going in either direction. To obtain the *unique* routes, we use an `if`-clause in a "hacky" way by only accepting routes where the first node has a smaller value than the last. Thus, we keep, for example, `(1, 2, 3)` and discard `(3, 2, 1)`.

In [66]:
for permutation in itertools.permutations(numbers):
    if permutation[0] < permutation[-1]:
        print(permutation)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)


In order to compare `Place`s as numbers, we would have to implement, among others, the `.__eq__()` special method. Otherwise, we get a `TypeError`.

In [67]:
Place(arrival, client=api) < Place(departure, client=api)

TypeError: '<' not supported between instances of 'Place' and 'Place'

As a quick and dirty solution, we use the `.location` property on a `Place` to do the comparison.

In [68]:
Place(arrival, client=api).location < Place(departure, client=api).location

False

As the code cell below shows, combining [permutations() <img height="12" style="display: inline-block" src="../static/link/to_py.png">](https://docs.python.org/3/library/itertools.html#itertools.permutations) with an `if`-clause results in the correct number of routes to be looped over.

In [69]:
sum(
    1
    for route in itertools.permutations(places)
    if route[0].location < route[-1].location
)

181440

To implement the brute force algorithm, we split the logic into two methods.

First, we create an `.evaluate()` method that takes a `route` argument that is a sequence of `Place`s and returns the total distance of the route. Internally, this method uses the `.distances` property repeatedly, which is why we built in caching above.

**Q32**: Finish the `.evaluate()` method as described!

Second, we create a `.brute_force()` method that needs no arguments. It loops over all possible routes to find the shortest. As the `start` and `end` of a route are fixed, we only need to look at `permutation`s of inner nodes. Each `permutation` can then be traversed in a forward and a backward order. `.brute_force()` enables method chaining as well.

**Q33**: Finish the `.brute_force()` method as described!

### The `Map` Class (continued): Brute Forcing the TSP

In [70]:
class Map:
    """A map with plotting and routing capabilities."""

    # answers from above
    def __init__(self, name, center, start, end, places, initial_zoom=12):
        """Create a new map.

        Args:
            name (str): name of the map
            center (float, float): coordinates of the map's center
            start (Place): start of the tour
            end (Place): end of the tour
            places (iterable of Places): the places to be visitied
            initial_zoom (integer): zoom level according to folium's
                specifications; defaults to 12
        """
        self.name = name
        self._start = start
        self._end = end
        self._places = places
        self._map = folium.Map(location=center, zoom_start=initial_zoom)
        self._distances = None

        self.add_marker(start.as_marker(color="green"))
        self.add_marker(end.as_marker(color="red"))
        for place in places:
            self.add_marker(place.as_marker())

    def __repr__(self):
        return f"<Map: {self.name}>"

    def show(self):
        """Return a folium.Map representation of the map."""
        return self._map

    def add_marker(self, marker):
        """Add a marker to the map.

        Args:
            marker (folium.Marker): marker to be put on the map
        """
        marker.add_to(self._map)
        return self

    @property
    def distances(self):
        """Return a dict with the pairwise distances of all places.

        Implementation note: The results of the calculations are cached.
        """
        if not self._distances:
            distances = {}
            all_pairs = itertools.combinations(
                [self._start, self._end] + self._places, r=2,
            )
            for origin, destination in all_pairs:
                distance = origin.distance_to(destination)
                distances[origin, destination] = distance
                distances[destination, origin] = distance
            self._distances = distances

        return self._distances

    # answer to Q32
    def evaluate(self, route):
        """Calculate the total distance of a route.

        Args:
            route (sequence of Places): the ordered nodes in a tour

        Returns:
            cost (int)
        """
        cost = 0
        # Loop over all pairs of consecutive places.
        origin = route[0]
        for destination in route[1:]:
            cost += self.distances[origin, destination]
            origin = destination

        return cost

    # answer to Q33
    def brute_force(self):
        """Calculate the shortest route by brute force.

        The route is plotted on the folium.Map.
        """
        # Assume a very high cost to begin with.
        min_cost = 999_999_999
        best_route = None

        # Loop over all permutations of the intermediate nodes to visit.
        for permutation in itertools.permutations(self._places):
            # Skip redundant permutations.
            if permutation[0].location > permutation[-1].location:
                continue
            # Travel through the routes in both directions.
            for route in (permutation, permutation[::-1]):
                # Add start and end to the route.
                route = (self._start, *route, self._end)
                # Check if a route is cheaper than all routes seen before.
                cost = self.evaluate(route)
                if cost < min_cost:
                    min_cost = cost
                    best_route = route

        # Plot the route on the map
        folium.PolyLine(
            [x.location for x in best_route],
            color="orange", weight=3, opacity=1
        ).add_to(self._map)

        return self

**Q34**: Find the best route for our tourist by executing the code cells below!

In [71]:
berlin = Map(
    "Berlin",
    center=(52.4915154, 13.4066838),
    start=Place(arrival, client=api).sync_from_google(),
    end=Place(departure, client=api).sync_from_google(),
    places=places,
    initial_zoom=12,
)

In [72]:
berlin.brute_force().show()